In [1]:
import numpy, sympy, scipy
from matplotlib import pyplot
import pandas
from sympy import oo, symbols, pprint
#함수 정의.
from scipy.stats import norm
import pickle

In [6]:
from sklearn.linear_model import LinearRegression

In [3]:
def win_pos_rate(pos, c1, c2, wl_d): #위치, 챔피언1, 챔피언2 입력 포지션에 대해 c1>c2 승률
    win_cnt = 0
    lose_cnt = 0
    for dat in wl_d: # 데이터에 대해 for문 돌리기
        if dat[1][pos] == c1 and dat[0][pos] == c2:
            win_cnt +=1
        elif dat[0][pos] == c1 and dat[1][pos] == c2:
            lose_cnt +=1
    return (win_cnt/(win_cnt+lose_cnt), win_cnt, lose_cnt) if win_cnt+lose_cnt !=0 else (0.5, 0,0)
        

In [8]:
def win_count(myteam, oppteam=None, wl_d=None): #상대승률 함수
    cnt_win = 0
    cnt_lose = 0
    if oppteam == None:
        for ele in wl_d:
            if not any(ele[1]- numpy.array(myteam)):
                cnt_win +=1
            elif not any(ele[0]- numpy.array(myteam)):
                cnt_lose +=1
        
    else:
        for ele in wl_d:
            if not any(ele[1]- numpy.array(myteam)) and not any(ele[0]- numpy.array(oppteam)):
                cnt_win +=1
            elif not any(ele[0]- numpy.array(myteam)) and not any(ele[1]- numpy.array(oppteam)):
                cnt_lose +=1
                
    return cnt_win, cnt_lose

In [13]:
def probe_test(mu =0, nu =0):
    x = numpy.arange(-10,10,1)
    res0 =0
    node = [ -numpy.sqrt(1/9*(5+2*numpy.sqrt(10/7))), 
            -numpy.sqrt(1/9*(5-2*numpy.sqrt(10/7))), 
            0, 
            numpy.sqrt(1/9*(5-2*numpy.sqrt(10/7))), 
            numpy.sqrt(1/9*(5+2*numpy.sqrt(10/7))) ]
    weight = [ 0.3*(0.7+5*numpy.sqrt(0.7))/(2+5*numpy.sqrt(0.7)) , 
              0.3*(-0.7+5*numpy.sqrt(0.7))/(-2+5*numpy.sqrt(0.7)) , 
              128/225, 
              0.3*(-0.7+5*numpy.sqrt(0.7))/(-2+5*numpy.sqrt(0.7)), 
              0.3*(0.7+5*numpy.sqrt(0.7))/(2+5*numpy.sqrt(0.7)) ]
    for xval in range(x.size):
        for yval in range(xval+1):
            res1, res2 =0, 0
            for c in range(5):
                res1 += 0.5*weight[c]*norm.pdf(0.5*node[c]+x[xval]+0.5 -mu)
            for d in range(5):
                res2 += 0.5*weight[d]*norm.pdf(0.5*node[d]+x[yval]+0.5 -nu)
            res0 += 0.5*res1*res2 if xval == yval else res1*res2
    
    return res0
            

            

In [2]:
#Gauss Quadrature 방식 사용. 
node = [ -numpy.sqrt(1/9*(5+2*numpy.sqrt(10/7))), -numpy.sqrt(1/9*(5-2*numpy.sqrt(10/7))), 0, numpy.sqrt(1/9*(5-2*numpy.sqrt(10/7))), numpy.sqrt(1/9*(5+2*numpy.sqrt(10/7))) ]
weight = [ 0.3*(0.7+5*numpy.sqrt(0.7))/(2+5*numpy.sqrt(0.7)) , 0.3*(-0.7+5*numpy.sqrt(0.7))/(-2+5*numpy.sqrt(0.7)) , 128/225, 0.3*(-0.7+5*numpy.sqrt(0.7))/(-2+5*numpy.sqrt(0.7)), 0.3*(0.7+5*numpy.sqrt(0.7))/(2+5*numpy.sqrt(0.7)) ]

In [3]:
x = numpy.arange(-10,10,1)
res = 0
for xval in x:
    for c in range(5):
        res += 0.5*weight[c]*norm.pdf(0.5*node[c]+xval+0.5)
print(res)


0.9999999999998352


In [4]:
norm.cdf(10)-norm.cdf(-10)

1.0

In [9]:
def probe_check(mu):
    nox = numpy.linspace(-0.4, 0.4, 100)
    return sum([probe_test(mu, nu) for nu in nox])/100

In [7]:
probe_test(-0.25,0)

0.4352222294080901

In [10]:
#-0.4~0.4 사이 10행 5열 행렬 출력. 강약의 계수 파악하기 위해서...
Xck = numpy.array([0]) #
while Xck.size>0:
    Xrand = (numpy.random.random((6,5))-0.5)*0.8
    Xrsum = Xrand.sum(axis=0).reshape(1,5)
    Xrand = Xrand - Xrsum/10 #합 0이 되게...
    Xck = Xrand[numpy.abs(Xrand)>0.4] #절대값 0.4 이상인 애들만 추려낸다. 만약 존재하면 다시 Xrand 생성

print(Xrand)

[[-0.17713161  0.26892904  0.26990395 -0.23573857 -0.36932229]
 [ 0.01530439 -0.26881778  0.03699475 -0.11236099 -0.35740223]
 [ 0.09217969  0.17643758  0.27351013 -0.24770681  0.07851897]
 [ 0.13586381  0.30184666 -0.24227742  0.22778853  0.26698483]
 [-0.11680889 -0.33148407 -0.32203253 -0.11360777  0.20844917]
 [-0.08068001 -0.15804065 -0.19224768  0.29779317  0.24239999]]


In [11]:
# 
Xrm =numpy.zeros((6,5,100) ) # 전적 숫자
Xrw =numpy.zeros((6,5,100) ) #승리 숫자
Xrl = numpy.zeros((6,5,100)) #승률 평균 구하기 위한 일종의 텐서.

for j in range(100):
    wl_d = [] #이기고 진 정보 저장
    for i in range(1000000):
        X1 = numpy.random.randint(0, 6, (5))
        X2 = numpy.random.randint(0, 6, (5))
        while all(X1-X2) == False: # 모두 다를 때까지 X1, X2 뽑는다.
            X1 = numpy.random.randint(0, 6, (5))
            X2 = numpy.random.randint(0, 6, (5))

        Y1 = numpy.random.randn(5)
        Y1 += numpy.array([ Xrand[X1[i],i] for i in range(5)]) # y1 값 할당
        Y2 = numpy.random.randn(5)
        Y2 += numpy.array([ Xrand[X2[i],i] for i in range(5)]) #Y2 값 할당
        
        Xrm[X1, range(5),j] +=1 #Xrm 매치 X1 추가
        Xrm[X2, range(5),j] +=1 #Xrm 매치 X2 추가

        if Y1.sum() > Y2.sum():
            Xrw[X1, range(5),j] +=1 #승자는 X1 조합
            wl_d.append({0:X2, 1:X1})
        elif Y1.sum() < Y2.sum():
            Xrw[X2, range(5),j] +=1 #승자는 X2 조합
            wl_d.append({0:X1, 1:X2})
        
        
    print(f'{j+1}번째 데이터 수집 완료!')
    with open(f'wl_d_{j}.pickle', 'wb') as pk:
        pickle.dump(wl_d, pk)
    
print('데이터 수집 모두 완료!')

1번째 데이터 수집 완료!
2번째 데이터 수집 완료!
3번째 데이터 수집 완료!
4번째 데이터 수집 완료!
5번째 데이터 수집 완료!
6번째 데이터 수집 완료!
7번째 데이터 수집 완료!
8번째 데이터 수집 완료!
9번째 데이터 수집 완료!
10번째 데이터 수집 완료!
11번째 데이터 수집 완료!
12번째 데이터 수집 완료!
13번째 데이터 수집 완료!
14번째 데이터 수집 완료!
15번째 데이터 수집 완료!
16번째 데이터 수집 완료!
17번째 데이터 수집 완료!
18번째 데이터 수집 완료!
19번째 데이터 수집 완료!
20번째 데이터 수집 완료!
21번째 데이터 수집 완료!
22번째 데이터 수집 완료!
23번째 데이터 수집 완료!
24번째 데이터 수집 완료!
25번째 데이터 수집 완료!
26번째 데이터 수집 완료!
27번째 데이터 수집 완료!
28번째 데이터 수집 완료!
29번째 데이터 수집 완료!
30번째 데이터 수집 완료!
31번째 데이터 수집 완료!
32번째 데이터 수집 완료!
33번째 데이터 수집 완료!
34번째 데이터 수집 완료!
35번째 데이터 수집 완료!
36번째 데이터 수집 완료!
37번째 데이터 수집 완료!
38번째 데이터 수집 완료!
39번째 데이터 수집 완료!
40번째 데이터 수집 완료!
41번째 데이터 수집 완료!
42번째 데이터 수집 완료!
43번째 데이터 수집 완료!
44번째 데이터 수집 완료!
45번째 데이터 수집 완료!
46번째 데이터 수집 완료!
47번째 데이터 수집 완료!
48번째 데이터 수집 완료!
49번째 데이터 수집 완료!
50번째 데이터 수집 완료!
51번째 데이터 수집 완료!
52번째 데이터 수집 완료!
53번째 데이터 수집 완료!
54번째 데이터 수집 완료!
55번째 데이터 수집 완료!
56번째 데이터 수집 완료!
57번째 데이터 수집 완료!
58번째 데이터 수집 완료!
59번째 데이터 수집 완료!
60번째 데이터 수집 완료!
61번째 데이터 수집 완료!
62번째 데이터 수집 완료!
63번째 데이터 수집 완료!
6

In [ ]:
win_rate = []
res =[0]*100
res1 = [0]*100

for j in range(100):
    with open(f'wl_d_{j}.pickle', 'rb') as pk:
        wl_d = pickle.load(pk)
        win_rate.append(Xrw[:,:,j]/Xrm[:,:,j])
    
    res[j] = [0]*5
    res1[j] = [0]*5
    for i in range(5):
        WL_M = numpy.ones((6,6))*0.5
        for t in range(6):
            for u in range(6):
                WL_M[t,u] = win_pos_rate(i, t, u, wl_d)[0]
        res[j][i] = pandas.DataFrame(data=WL_M, columns=list(range(6)), index=list(range(6)))
        WL_M1 = numpy.ones((6,6))*0.5
        for t in range(6):
            for u in range(6):
                WL_M1[t,u] = probe_test(Xrand[t,i], Xrand[u,i])
        res1[j][i] = pandas.DataFrame(data=WL_M1, columns=list(range(6)), index=list(range(6)))
        
    

In [ ]:
#이긴 조합, 진 조합 정보 저장.
wl_d[0:100]

In [ ]:
res = [0,0,0,0,0]
for i in range(5):
    print(f'포지션 {i} 챔피언별 승률 : ')
    WL_M = numpy.ones((6,6))*0.5
    for j in range(6):
        for k in range(6):
            WL_M[j,k] = win_pos_rate(i, j, k)[0]
    res[i] = pandas.DataFrame(data=WL_M, columns=list(range(6)), index=list(range(6)))
    print(res[i])

In [ ]:
res1 = [0,0,0,0,0]
for i in range(5):
    print(f'포지션 {i} 챔피언별 이론승률: ')
    WL_M1 = numpy.ones((6,6))*0.5
    for j in range(6):
        for k in range(6):
            WL_M1[j,k] = probe_test(Xrand[j,i], Xrand[k,i])
    res1[i] = pandas.DataFrame(data=WL_M1, columns=list(range(6)), index=list(range(6)))
    print(res1[i])

In [ ]:


fig, ax = pyplot.subplots(6,5, figsize=(15,22))
fig.tight_layout()
for i in range(6):
    for j in range(5):
        ax[i][j].plot(range(6), [numpy.mean((Xrw/Xrm), axis=2)[i,j]]*6, 'g:' )
        ax[i][j].plot(range(6), [0.5]*6, 'r:' )
        ax[i][j].plot(range(6), numpy.array([res[k][j].iloc[i,:] for k in range(100)]).mean(), 'b-^')
        ax[i][j].set_title(f'Win rate of \nChampion {i} \nwith position {j}')
pyplot.show()
pyplot.savefig('champ_win_rate1.png',dpi=250)

In [ ]:
#실제 조합별 승률 측정모델
wl_d[42130][1]

In [ ]:

xdats = []
for i in range(5):    
    xdat = numpy.array([[0]*6+[0.5]]) #데이터 수집
    for t in range(6):
        for u in range(6):
            if t !=u:
                xnew = numpy.zeros(7).reshape(1,-1)
                xnew[0,j] = 1
                xnew[0,k] =-1
                xnew[0,6] = numpy.array([res[k][i].iloc[t,u] for k in range(100)]).mean() 
                xdat = numpy.concatenate((xdat, xnew), axis=0)
    xdats.append(xdat)
            

In [ ]:
Tr_x = [xdats[i][:, 0:6] for i in range(5)]
Tr_y = [xdats[i][:,6] for i in range(5)]

In [ ]:
Tr_x[0].shape

In [ ]:
LR = [0,0,0,0,0]
for i in range(5):
    LR[i] = LinearRegression()
    LR[i].fit(Tr_x[i], Tr_y[i])
    print(i)
    print(LR[i].coef_, LR[i].intercept_)

In [ ]:
def predict_win_rate(myteam, oppteam):
    win =0.5
    for i in range(5):
        dr = numpy.zeros(6).reshape(1,-1)
        dr[0, [myteam[i], oppteam[i]]] = [1,-1]
        win += LR[i].predict(dr)-0.5
    return win
        

In [ ]:
res00 = [numpy.zeros((6,6))]*5
for i in range(5):
    for t in range(6):
        for u in range(6):
            res00[i][t,u] = [res[k][i][t,u] for k in range(100)]
            
def predict_win_rate(myteam, oppteam): #linear requression
    win =0.5
    for i in range(5):
        dr = numpy.zeros(6).reshape(1,-1)
        dr[0, [myteam[i], oppteam[i]]] = [1,-1]
        win += LR[i].predict(dr)-0.5
    return win

def predict_win_rate2(myteam, oppteam):
    myteam_prod = 1
    oppteam_prod = 1
    for i in range(5):
        myteam_prod *=res00[i].iloc[myteam[i], oppteam[i]]
        oppteam_prod *= res00[i].iloc[oppteam[i], myteam[i]]
    return (myteam_prod) / (myteam_prod + oppteam_prod)
    

In [ ]:
predict_win_rate([1,2,0,4,1], [0,3,5,2,1])

In [ ]:
predict_win_rate2([1,2,0,4,1],[0,3,5,2,1])

In [ ]:
win_count([1,2,0,4,1],[0,3,5,2,1])